In [4]:
import torch
import torch.nn as nn
import torchvision.datasets as dsets
import torchvision.transforms as transforms
from torch.autograd import Variable

In [5]:
# Hyper Parameters
num_epochs = 5
batch_size = 100
learning_rate = 0.001

In [8]:
# MNIST Dataset
train_dataset = dsets.MNIST(root = '../data/',
                           train = True,
                           transform = transforms.ToTensor(),
                           download = True)
test_dataset = dsets.MNIST(root = '../data/',
                         train = False,
                         transform = transforms.ToTensor())

# Data Loader(Input Pipeline)
train_loader = torch.utils.data.DataLoader(dataset = train_dataset,
                                          batch_size = batch_size,
                                          shuffle = True)
test_loader = torch.utils.data.DataLoader(dataset = test_dataset,
                                         batch_size = batch_size,
                                         shuffle = False)

In [26]:
# CNN Model(2 conv layer)
class CNN(nn.Module):
    def __init__(self):
        super(CNN,self).__init__()
        self.layer1 = nn.Sequential(nn.Conv2d(1,16,kernel_size = 5, padding=2),
                                   nn.BatchNorm2d(16),
                                   nn.ReLU(),
                                   nn.MaxPool2d(2))
        self.layer2 = nn.Sequential(nn.Conv2d(16,32,kernel_size=5,padding=2),
                                  nn.BatchNorm2d(32),
                                  nn.ReLU(),
                                  nn.MaxPool2d(2))
        self.fc = nn.Linear(7*7*32,10)
        
    def forward(self,x):
        out = self.layer1(x)
        out = self.layer2(out)
        out = out.view(out.size(0),-1)
        out = self.fc(out)
        return out

In [27]:
cnn = CNN()

In [28]:
# Loss and Optimizer
criterion = nn.CrossEntropyLoss()
optimizer = torch.optim.Adam(cnn.parameters(),lr=learning_rate)

In [32]:
# Train the Model
for epoch in range(num_epochs):
    for i,(images,labels)in enumerate(train_loader):
        images = Variable(images)
        labels = Variable(labels)
        # Forward + Backward + Optimize
        optimizer.zero_grad()
        outputs = cnn(images)
        loss = criterion(outputs,labels)
        loss.backward()
        optimizer.step()
        
        if(i+1)%100 ==0:
            print('Epoch [%d/%d],Step [%d/%d],Loss:%.4f'
                 %(epoch+1,num_epochs,i+1,len(train_dataset)//batch_size,loss.data[0]))

Epoch [1/5],Step [100/600],Loss:0.0830
Epoch [1/5],Step [200/600],Loss:0.0848
Epoch [1/5],Step [300/600],Loss:0.1513
Epoch [1/5],Step [400/600],Loss:0.0175
Epoch [1/5],Step [500/600],Loss:0.0760
Epoch [1/5],Step [600/600],Loss:0.0429
Epoch [2/5],Step [100/600],Loss:0.0155
Epoch [2/5],Step [200/600],Loss:0.0778
Epoch [2/5],Step [300/600],Loss:0.0093
Epoch [2/5],Step [400/600],Loss:0.0052
Epoch [2/5],Step [500/600],Loss:0.0273
Epoch [2/5],Step [600/600],Loss:0.0261
Epoch [3/5],Step [100/600],Loss:0.0821
Epoch [3/5],Step [200/600],Loss:0.0691
Epoch [3/5],Step [300/600],Loss:0.0099
Epoch [3/5],Step [400/600],Loss:0.0063
Epoch [3/5],Step [500/600],Loss:0.0167
Epoch [3/5],Step [600/600],Loss:0.0326
Epoch [4/5],Step [100/600],Loss:0.0230
Epoch [4/5],Step [200/600],Loss:0.0206
Epoch [4/5],Step [300/600],Loss:0.0141
Epoch [4/5],Step [400/600],Loss:0.0094
Epoch [4/5],Step [500/600],Loss:0.0327
Epoch [4/5],Step [600/600],Loss:0.0459
Epoch [5/5],Step [100/600],Loss:0.0403
Epoch [5/5],Step [200/600

In [33]:
# Test the Model
cnn.eval() # Change model to 'eval' mode
correct = 0
total = 0
for images,labels in test_loader:
    images = Variable(images)
    outputs = cnn(images)
    _,predicted = torch.max(outputs.data,1)
    total += labels.size(0)
    correct += (predicted==labels).sum()

print('Test Accuracy of the model on the 10000 test images: %d %%'%(100*correct/total))
    

Test Accuracy of the model on the 10000 test images: 98 %
